# Aknowledgements

*This is a copy of [Vighnesh Birodkar's notebook](https://www.kaggle.com/code/vighneshbgoogle/iwildcam-visualize-instance-masks/) from iWildCam 2021, adapted for the 2022 competition.*

# About the notebook

This notebook visualized the accompanying instance masks for the iWildCam 2022 challenge. Instance masks are provided by the *DeepMAC* model from the paper ["The surprising impact of mask-head architecture on novel class segmentation"](https://arxiv.org/abs/2104.00613). DeepMAC was designed to produce accurate instance segmentation masks for unseen classes. For this challenge, we use the best model from the paper (trained on all of COCO) and combine it with detections from [MegaDetector](https://github.com/microsoft/CameraTraps/blob/master/megadetector.md). For each detection in MegaDetector, there is an accompanying instance mask. The format is described in detail [here](https://github.com/visipedia/iwildcam_comp#format-details).

# Imports and Definitions

In [ ]:
import json
import os
import random

import cycler
from matplotlib import colors
from matplotlib import pyplot as plt
import matplotlib.patches as patches
import numpy as np
from PIL import Image
import tensorflow as tf


COLOR_CYCLER = cycler.cycler(color=['tab:blue', 'tab:green', 'tab:orange',
                                    'tab:red', 'tab:purple'])


def read_image(path):
  with tf.io.gfile.GFile(path, 'rb') as f:
    return np.array(Image.open(f))


def read_json(path):
  with tf.io.gfile.GFile(path) as f:
    return json.load(f)


def create_detection_map(annotations):
  """Creates a dict mapping IDs to detections."""

  ann_map = {}
  for image in annotations['images']:
    ann_map[image['file'].split('/')[-1].rstrip('.jpg')] = image['detections']
  return ann_map


def image_name_to_id(name):
  return name.rstrip('.jpg')


def plot_image_annotation(image, detection_annotations, categories,
                          instance_id_image, show_label=True):
  """Plot boxes and mask annotations for a given image.

  Args:
    image: An image array of shape [H, W, 3]
    detection_annotations: A list of detections. Each detection is a dict
      containing the keys 'category', 'bbox' and 'conf'.
    categories: A dict mapping category IDs to names.
    instance_id_image: An array of shape [H, W] containing the instance ID
      at each pixel. IDs are expected to be 1-indexed, with 0 reserved for
      the background.
    show_label: bool, whether or not to show the label of each object
      in the plot.
  """

  fig, ax = plt.subplots(figsize=(12, 9))
  image_height, image_width = image.shape[:2]

  ax.imshow(image)

  cycle_iter = COLOR_CYCLER()
  for i, annotation in enumerate(detection_annotations):
    xmin, ymin, width, height = annotation['bbox']
    xmin *= image_width
    ymin *= image_height
    width *= image_width
    height *= image_height

    color = next(cycle_iter)['color']
    rect = patches.Rectangle((xmin, ymin), width, height,
                             linewidth=3, edgecolor=color, facecolor='none')
    ax.add_patch(rect)
    label = '{}:{:.2f}'.format(categories[annotation['category']],
                               annotation['conf'])
    if show_label:
      ax.text(xmin, ymin - 5, label, fontsize=30, color='white',
              bbox=dict(boxstyle='square,pad=0.0', facecolor=color, alpha=0.75,
                        ec='none'))
    
    r, g, b, _ = colors.to_rgba(color)
    color_array = np.array([r, g, b]).reshape(1, 1, 3)
    color_image = np.ones((image_height, image_width, 3)) * color_array
    mask = (instance_id_image == (i + 1)).astype(np.float32)[:, :, np.newaxis]
    color_mask = np.concatenate([color_image, mask], axis=2)

    ax.imshow(color_mask, alpha=0.5)

# Load metadata information

In [ ]:
# Either train or test directory
IMAGES_DIR = "/kaggle/input/iwildcam2022-fgvc9/train/train"
BOX_ANNOTATION_FILE = "/kaggle/input/iwildcam2022-fgvc9/metadata/metadata/iwildcam2022_mdv4_detections.json"
MASKS_DIR = "/kaggle/input/iwildcam2022-fgvc9/instance_masks/instance_masks"

images = tf.io.gfile.listdir(IMAGES_DIR)

# The annotations file contains annotations for all images in train and test
annotations = read_json(BOX_ANNOTATION_FILE)
detection_map = create_detection_map(annotations)
image_ids = list(detection_map.keys())


# Visualize a random sample

Sample a random image from `IMAGES_DIR` and visualize its instances if there are any.

In [ ]:
image_name = random.choice(images)
image_path = os.path.join(IMAGES_DIR, image_name)
image_id = image_name_to_id(image_name)
mask_path = os.path.join(MASKS_DIR, f'{image_id}.png')

if image_id not in detection_map:
  print(f'Image {image_name} is missing detection data.')
elif len(detection_map[image_id]) == 0:
  print(f'There are no detected objects in the image {image_name}.')
elif not tf.io.gfile.exists(mask_path):
  print(f'No mask found for {image_id}')
else:
  detection_annotations = detection_map[image_name_to_id(image_name)]
  image = read_image(image_path)
  instance_id_image = read_image(mask_path)
  plot_image_annotation(image, detection_annotations,
                        annotations['detection_categories'], instance_id_image)